In [0]:
#https://youtu.be/8DXUcJtvR0w
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
spark_dis_AQE=SparkSession.builder.appName("Data_skewness_before_enabling_AEQ").getOrCreate()

spark_dis_AQE.conf.set("spark.sql.adaptive.enabled", False)
#spark_dis_AQE.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)
#spark_dis_AQE.conf.set("spark.sql.shuffle.partitions", 3)


In [0]:
df1 = spark_dis_AQE.range(1000) \
    .withColumn("id", lit("x"))

extravalues = spark_dis_AQE.range(4)\
    .withColumn("id", lit("y"))

moreextravalues = spark_dis_AQE.range(4)\
    .withColumn("id", lit("z"))


df1 = df1.union(extravalues).union(moreextravalues)


#########

df2=spark_dis_AQE.range(1000) \
    .withColumn("id", lit("x"))
extravalues1 = spark_dis_AQE.range(1)\
    .withColumn("id", lit("y"))

moreextravalues1 = spark_dis_AQE.range(1)\
    .withColumn("id", lit("z"))

df2 = df2.union(extravalues1).union(moreextravalues1)





In [0]:
f_df1=df1.join(df2,'id').select("id").distinct()

f_df1.display(5)
f_df1.rdd.getNumPartitions()

Out[4]: 200

id x z y

In [0]:
f_df1.explain()

== Physical Plan ==
*(9) HashAggregate(keys=[id#43], functions=[])
+- Exchange hashpartitioning(id#43, 200), ENSURE_REQUIREMENTS, [id=#224]
   +- *(8) HashAggregate(keys=[id#43], functions=[])
      +- *(8) BroadcastHashJoin [id#43], [id#57], LeftSemi, BuildRight, false
         :- *(8) HashAggregate(keys=[id#43], functions=[])
         :  +- Union
         :     :- *(1) Project [x AS id#43]
         :     :  +- *(1) Range (0, 1000, step=1, splits=8)
         :     :- *(2) Project [y AS id#47]
         :     :  +- *(2) Range (0, 4, step=1, splits=8)
         :     +- *(3) Project [z AS id#51]
         :        +- *(3) Range (0, 4, step=1, splits=8)
         +- BroadcastExchange HashedRelationBroadcastMode(ArrayBuffer(input[0, string, false]),false), [id=#219]
            +- *(7) HashAggregate(keys=[id#57], functions=[])
               +- Union
                  :- *(4) Project [x AS id#57]
                  :  +- *(4) Range (0, 1000, step=1, splits=8)
                  :- *(5) Project 

In [0]:
## AQE

spark_dis_AQE.conf.get("spark.sql.adaptive.enabled")

Out[5]: 'false'

In [0]:
spark_dis_AQE.conf.set("spark.sql.adaptive.enabled",True)

In [0]:
spark_dis_AQE.conf.get("spark.sql.adaptive.coalescePartitions.enabled")

Out[7]: 'true'

In [0]:
spark_dis_AQE.conf.set("spark.sql.adaptive.coalescePartitions.enabled",True)

In [0]:
f_df2=df1.join(df2,'id').select("id").distinct()

f_df2.display(5)
f_df2.rdd.getNumPartitions()

Out[9]: 1

id x y z

In [0]:
f_df2.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=true
+- == Final Plan ==
   *(9) HashAggregate(keys=[id#43], functions=[])
   +- CustomShuffleReader coalesced
      +- ShuffleQueryStage 1, Statistics(sizeInBytes=384.0 B, rowCount=16, isRuntime=true)
         +- Exchange hashpartitioning(id#43, 200), ENSURE_REQUIREMENTS, [id=#846]
            +- *(8) HashAggregate(keys=[id#43], functions=[])
               +- *(8) BroadcastHashJoin [id#43], [id#57], LeftSemi, BuildRight, false
                  :- *(8) HashAggregate(keys=[id#43], functions=[])
                  :  +- Union
                  :     :- *(5) Project [x AS id#43]
                  :     :  +- *(5) Range (0, 1000, step=1, splits=8)
                  :     :- *(6) Project [y AS id#47]
                  :     :  +- *(6) Range (0, 4, step=1, splits=8)
                  :     +- *(7) Project [z AS id#51]
                  :        +- *(7) Range (0, 4, step=1, splits=8)
                  +- BroadcastQueryStage 0, Statistics(size